# Important Libraries

In [1]:
# Initialization commands
%matplotlib inline

# Important libraries
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import seaborn as sns

import numpy.random as npr # random sampling functions
import scipy.stats as sps # statistical functions
import matplotlib.pyplot as plt # Matlab plotting framework
import matplotlib.mlab as mlab
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import time

# Important functions
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from mpl_toolkits.mplot3d import Axes3D
from numpy import array
from scipy.io import loadmat
from multiprocessing.dummy import Pool as ThreadPool
from scipy.cluster.vq import vq, kmeans, whiten
from matplotlib import colors as mcolors
colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)

# Misc settings
sns.set(context="poster", style="whitegrid")

import warnings
warnings.filterwarnings('ignore')

# Functions

## Model-specifc Functions:

### Log-posterior

In [2]:
def logPost(theta,zHat,oHat):
    
    mm = oHat.shape[1]
    kk = zHat.shape[1]
    nn = oHat.shape[0]
    
    # form the covariance matrix
    covM = tril2cov(theta[-mm*(mm+1)/2:],mm)
    
    # form the beta matrix
    betaM = np.reshape(theta[:mm*kk],(kk,mm))
    arg = np.transpose(oHat - np.dot(zHat,betaM))
    
    # compute the log-likelihood at this location in phase space
    logLike = -0.5*nn*np.log(np.linalg.det(covM)) - 0.5*np.trace(np.linalg.solve(covM, np.dot(arg,arg.T)))
    
    return logLike

### Log-prior: Normal Distribution

In [3]:
def priorNorm(x,mu,sigma): 
    
    return np.sum(- 0.5*np.log(2*np.pi) - np.log(sigma) - ((x - mu)**2)/(2*(sigma**2)))

### Covariance Matrix
Cholesky decomposition of a Hermitian positive-definite matrix is defined as $\Sigma = LDL^T$, where $L$ is any real-valued lower unit triangular matrix and $D$ is a diagonal matrix with positive elements. We allow the MVN proposal distribution to populate the elements of $L$ in such a way that all proposals provided by the jumping kernel are valid. The code below manipulates these raw MVN proposal locations to ensure the resulting $\Sigma$ is always a Hermitian positive-definite matrix.

$$\theta^* \sim\ MVN(\theta,\lambda I)$$

$$\Sigma = LDL^T = \left[ \begin{array}{cccc}
1 & 0 & 0 \\
\theta_{4} & 1 & 0 \\
\theta_{5} & \theta_{6} & 1 \\ \end{array} \right]
\left[ \begin{array}{cccc}
\exp(\theta_{1}) & 0 & 0 \\
0 & \exp(\theta_{2}) & 0 \\
0 & 0 & \exp(\theta_{3}) \\ \end{array} \right]
\left[ \begin{array}{cccc}
1 & \theta_{4} & \theta_{5} \\
0 & 1 & \theta_{6} \\
0 & 0 & 1 \\ \end{array} \right] =
\left[ \begin{array}{cccc}
\sigma_{1}^2 & \rho_{12}\sigma_{1}\sigma_{2} & \rho_{13}\sigma_{1}\sigma_{3} \\
\rho_{12}\sigma_{1}\sigma_{2} & \sigma_{2}^2 & \rho_{23}\sigma_{2}\sigma_{3} \\
\rho_{13}\sigma_{1}\sigma_{3} & \rho_{23}\sigma_{2}\sigma_{3} & \sigma_{3}^2 \\ \end{array} \right]$$

In [4]:
def tril2cov(x,m):
    
    D = np.diag(np.exp(x[:m]))
    L = np.zeros((m,m))
    L[np.tril_indices(m,-1)] = x[-(len(x)-m):]
    L[np.diag_indices(m)] = np.ones(m) 
    
    return np.dot(np.dot(L,D),L.T)

### Adaptive Metropolis Sampler with MVN Block Updates 

In [5]:
def mcmc(ZH,OH,nn,NN,aa):
    
    # preallocate standard normal MVN matrix of proposals
    mvnJ = npr.multivariate_normal(np.zeros((1,nn)).ravel(),np.eye(nn),NN)
    mvnN = np.reshape(np.apply_along_axis(np.linalg.norm,1,mvnJ),(-1,1))
    mvnJ = mvnJ/mvnN
    
    # initialize the chains 
    pw = np.zeros((1,NN)).ravel()
    ns = np.zeros((1,NN)).ravel()
    theta = np.zeros((NN,nn))
    rndVal = np.log(npr.rand(1,NN)).ravel()
    jj = 0
    
    for i in range(NN):
        
        if np.less(jj,1):
            
            jumpVar = npr.randn()
            thetaHat = npr.multivariate_normal(np.zeros((1,nn)).ravel(),np.eye(nn))
        
        pw[i] = 2*jumpVar
        thetaStar = np.exp(jumpVar)*mvnJ[i] + thetaHat
        logOdds = logPost(thetaStar,ZH,OH) - logPost(thetaHat,ZH,OH)
        
        if np.less(rndVal[i],logOdds):
            
            thetaHat = thetaStar
            jj += 1
        
        if np.greater(jj,0):
            
            jmpDev = np.log(jj) - np.log(i+1) - np.log(aa)
            jmpRel = (NN/4 - 1e3)*((i+1)/NN) + 1e3
            jumpVar = jumpVar + jmpDev/jmpRel
        
        ns[i] = jj/(i+1)
        theta[i] = thetaHat
        
    ns = ns.flatten().tolist()
    pw = pw.flatten().tolist()
    
    return theta,ns,pw

## Helper Functions:

### Data Standardization: Z-Scores
The computational behavior of MLE and MCMC methods is improved when working with data has been shifted and scaled to have $\mu=0$ and $\sigma^2=1$ along each dimension of the data vector.

In [6]:
# center and/or scale data to have mean = 0 and variance = 1

def zscore(data, mode=0):
    
    mu = np.mean(data)
    sig = np.std(data)
    
    if mode == 0:
        
        z = (data - mu)/sig
        
    else:
        
        z = data/sig
    
    return z,mu,sig

In [7]:
# provide an inverse z-score transformation

def zreturn(data, mu, sig, dim=0):
    
    return sig[dim]*data + mu[dim]

# Bayesian Analysis

In [8]:
cityNames = ['HAR','GRI','RNO','TVC','SEA','JAN','BIS','BNA','DFW']
# cityNames = ['EYW']

# Data file
dfTmax = pd.read_pickle('Tmax.pkl')
dfTmin = pd.read_pickle('Tmin.pkl')
dfVmax = pd.read_pickle('Vmax.pkl')
dfClimo = pd.read_pickle('Climo.pkl')

chainNum = 0
mdl = 'ARCN'

burnVal = 1e6 # burn-in period
warmVal = burnVal # number of post-burn-in samples
thinVal = warmVal/100 # number target samples retained post-burn-in

accT = 0.234
lags = 100
    
thinFrac = thinVal/warmVal; # fraction retained from warmed samples
N = warmVal + burnVal

ARList = ['ARN1', 'ARN2', 'ARN3', 'ARN4', 'ARN5', 'ARN6','ARP1', 'ARP2', 'ARP3', 'ARP4', 'ARP5', 'ARP6']
MBList = ['MBN1', 'MBN2', 'MBN3', 'MBN4', 'MBN5', 'MBN6','MBP1', 'MBP2', 'MBP3', 'MBP4', 'MBP5', 'MBP6']
SRList = ARList + MBList
fullList = ['ARCN','MBCN','OBS'] + SRList

In [9]:
t1 = time.clock()

for cIdx, cityStr in enumerate(cityNames):
    
    dfT = dfTmax[dfTmax['City']==cityStr]
    dft = dfTmin[dfTmin['City']==cityStr]
    dfV = dfVmax[dfVmax['City']==cityStr]

    # ubiquitous log-transformation of the data
    dfT.loc[:,fullList] = np.log(dfT.loc[:,fullList])
    dft.loc[:,fullList] = np.log(dft.loc[:,fullList])
    dfV.loc[:,fullList] = np.log(dfV.loc[:,fullList])
    
    dfT = dfT.reindex_axis(sorted(dfT.columns), axis=1)
    dft = dft.reindex_axis(sorted(dft.columns), axis=1)
    dfV = dfV.reindex_axis(sorted(dfV.columns), axis=1)

    T = array(dfT.loc[dfT['Type']=='Train', 'OBS']).reshape(-1,1)
    t = array(dft.loc[dft['Type']=='Train', 'OBS']).reshape(-1,1)
    V = array(dfV.loc[dfV['Type']=='Train', 'OBS']).reshape(-1,1)

    obs = np.concatenate((T,t,V),axis=1)
    
    del T, t, V
    
    T = array(dfT.loc[dfT['Type']=='Train', mdl]).reshape(-1,1)
    t = array(dft.loc[dft['Type']=='Train', mdl]).reshape(-1,1)
    V = array(dfV.loc[dfV['Type']=='Train', mdl]).reshape(-1,1)

    data = np.concatenate((T,t,V),axis=1)

    obs = pd.DataFrame(obs)
    data = pd.DataFrame(data)

    X,xMu,xStd = zscore(data)
    O,oMu,oStd = zscore(obs)
    
    xMu = array(xMu.tolist()).reshape(1,-1)
    xStd = array(xStd.tolist()).reshape(1,-1)
    oMu = array(oMu.tolist()).reshape(1,-1)
    oStd = array(oStd.tolist()).reshape(1,-1)
    
    X['D'] = 1
    cols = X.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    X = X[cols]

    X = X.as_matrix()
    O = O.as_matrix()
    mm = O.shape[1]
    kk = X.shape[1]
    nParm = kk*mm + mm*(mm+1)/2 # number of parameters

    th,ns,pw = mcmc(X,O,int(nParm),int(N),accT) 
    
    TH = pd.DataFrame(th)
    TH = TH.rename(columns = lambda x : 'B_' + str(x))
    TH['City'] = cityStr
    TH['Accept'] = ns
    TH['Width'] = pw
    TH['Proposal'] = TH.index.tolist()
    TH['Chain'] = chainNum
    
    if cIdx == 0:
    
        dfTH = TH.copy()
        XMU = xMu
        XSTD = xStd
        OMU = oMu
        OSTD = oStd
        
    else:
        
        XMU = np.concatenate((XMU,xMu))
        XSTD = np.concatenate((XSTD,xStd))
        OMU = np.concatenate((OMU,oMu))
        OSTD = np.concatenate((OSTD,oStd))
        dfTH = pd.concat([dfTH,TH], ignore_index=True)
    
    # del th, ns, pw, xMu, xStd, oMu, oStd, TH, X, O, cols
    
XMU = pd.DataFrame(XMU, columns = ['Tmax','Tmin','Vmax'])
XSTD = pd.DataFrame(XSTD, columns = ['Tmax','Tmin','Vmax'])
OMU = pd.DataFrame(OMU, columns = ['Tmax','Tmin','Vmax'])
OSTD = pd.DataFrame(OSTD, columns = ['Tmax','Tmin','Vmax'])

XMU['City'] = cityNames
XMU['Type'] = 'x'
XMU['Stat'] = 'mu'

XSTD['City'] = cityNames
XSTD['Type'] = 'x'
XSTD['Stat'] = 'std'

OMU['City'] = cityNames
OMU['Type'] = 'o'
OMU['Stat'] = 'mu'

OSTD['City'] = cityNames
OSTD['Type'] = 'o'
OSTD['Stat'] = 'std'

dfZS = pd.concat([XMU,XSTD,OMU,OSTD], ignore_index=True)

dfTH.to_pickle(mdl + '_params_' + str(chainNum) + '.pkl')
dfZS.to_pickle(mdl + '_zscores.pkl')

t2 = time.clock() - t1
tt = N*len(cityNames)/t2
s0 = 'Sampling time: ' + repr(t2/60) + '[min]'
s1 = 'Mean sampling rate: ' + repr(tt) + ' [proposals/sec]'
s2 = 'Mean sampling time per 10^6 proposals: ' + repr(1e6/(tt*60)) + ' [min]'
print(s0)
print(s1)
print(s2)

Sampling time: 98.14369243216699[min]
Mean sampling rate: 3056.74254315781 [proposals/sec]
Mean sampling time per 10^6 proposals: 5.45242735734261 [min]
